## Exercici 1
Descarrega el data set Airlines Delay: Airline on-time statistics and delay causes i carrega’l a un pandas Dataframe. Explora les dades que conté, i queda’t únicament amb les columnes que consideris rellevants.

In [2]:
import pandas as pd

df = pd.read_csv("DelayedFlights.csv", header=None, nrows=800000)
print(df)
print(df.columns)

df.columns = df.iloc[0]
df = df.iloc[1: , :]
print(df.columns)

df_seleccio = df[['Month', 'DayOfWeek', 'ArrDelay', 'DepDelay', 'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay', 'Origin', 'Dest', 'Distance', 'AirTime','UniqueCarrier']]
print(df_seleccio)

/Users/claradv/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (1,2,3,4,5,6,7,8,10,12,13,14,15,16,19,20,21,22,24,25,26,27,28,29) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


               0     1      2           3          4        5           6   \
0             NaN  Year  Month  DayofMonth  DayOfWeek  DepTime  CRSDepTime   
1             0.0  2008      1           3          4   2003.0        1955   
2             1.0  2008      1           3          4    754.0         735   
3             2.0  2008      1           3          4    628.0         620   
4             4.0  2008      1           3          4   1829.0        1755   
...           ...   ...    ...         ...        ...      ...         ...   
799995  2655507.0  2008      5           9          5   1320.0        1310   
799996  2655508.0  2008      5           9          5   1508.0        1435   
799997  2655509.0  2008      5           9          5   1758.0        1745   
799998  2655514.0  2008      5           9          5    711.0         640   
799999  2655515.0  2008      5           9          5   1631.0        1610   

             7           8              9   ...      20       2

## Exercici 2
Fes un informe complet del data set:

- Resumeix estadísticament les columnes d’interès
- Troba quantes dades faltants hi ha per columna
- Crea columnes noves (velocitat mitjana del vol, si ha arribat tard o no...)
- Taula de les aerolínies amb més endarreriments acumulats
- Quins són els vols més llargs? I els més endarrerits?
- Etc.

In [31]:
import numpy as np

# Dades faltants per columna
for column in df_seleccio:
    print('Columna:', column)
    print('Dades faltants:', df_seleccio[column].isna().sum(), '\n')
    # substiteixo els Nan per zeros per a poder operar amb aquests valors més endavant
    df_seleccio.loc[:, column] = df_seleccio[column].fillna(0)
    
    
# Canviar data type per a determinades columnes i així poder fer càlculs
df_seleccio.loc[:,'ArrDelay'] = df_seleccio['ArrDelay'].astype(float).astype(int)
df_seleccio.loc[:,'DepDelay'] = df_seleccio['DepDelay'].astype(float).astype(int)
df_seleccio.loc[:,'CarrierDelay'] = df_seleccio['CarrierDelay'].astype(float).astype(int)
df_seleccio.loc[:,'WeatherDelay'] = df_seleccio['WeatherDelay'].astype(float).astype(int)
df_seleccio.loc[:,'NASDelay'] = df_seleccio['NASDelay'].astype(float).astype(int)
df_seleccio.loc[:,'SecurityDelay'] = df_seleccio['SecurityDelay'].astype(float).astype(int)
df_seleccio.loc[:,'LateAircraftDelay'] = df_seleccio['LateAircraftDelay'].astype(float).astype(int)
df_seleccio.loc[:,'Distance'] = df_seleccio['Distance'].astype(float).round(2)
df_seleccio.loc[:,'AirTime'] = df_seleccio['AirTime'].astype(float).astype(int)


# Resum stadístic
for column in df_seleccio:
    print(df_seleccio[column].describe(), "\n")
    
    
# Crear columnas nuevas
df_seleccio.loc[:,'TotalDelay'] = df_seleccio['ArrDelay'] + df_seleccio['DepDelay'] + df_seleccio['CarrierDelay'] + df_seleccio['WeatherDelay'] + df_seleccio['NASDelay'] + df_seleccio['SecurityDelay'] + df_seleccio['LateAircraftDelay']
df_seleccio.loc[:,'AverageSpeed'] = (df_seleccio['Distance'] / df_seleccio['AirTime']).round(2)
df_seleccio.loc[:,'Delayed'] = np.where(df_seleccio['TotalDelay'] > 0, True, False)


# Taula de les aerolínies amb més endarreriments acumulats
aerolinies_amb_mes_endarreriments = df_seleccio.groupby(['UniqueCarrier']).size().reset_index(name='NumDelays')
aerolinies_amb_mes_endarreriments = aerolinies_amb_mes_endarreriments.sort_values(by=['NumDelays'], ascending=False).head(3)
print(aerolinies_amb_mes_endarreriments)


# Vols més llargs
data = [df_seleccio["Origin"], df_seleccio["Dest"], df_seleccio["AirTime"]]
headers = ["Origin", "Dest", "Duration"]
vols = pd.concat(data, axis=1, keys=headers)
vols_mes_llargs = vols.sort_values(by=['Duration'], ascending=False).head(3)
print(vols_mes_llargs)


# Vols més endarrerits
data = [df_seleccio["Origin"], df_seleccio["Dest"], df_seleccio["TotalDelay"]]
headers = ["Origin", "Dest", "Delay"]
vols = pd.concat(data, axis=1, keys=headers)
vols_mes_endarrerits = vols.sort_values(by=['Delay'], ascending=False).head(3)
print(vols_mes_endarrerits)


# Mesos amb més endarreriments
mesos_amb_mes_endarreriments = df_seleccio.groupby(['Month']).size().reset_index(name='NumDelays')
mesos_amb_mes_endarreriments.loc[:,'Month'] = pd.to_datetime(mesos_amb_mes_endarreriments['Month'], format='%m').dt.month_name().str.slice(stop=3)
mesos_amb_mes_endarreriments = mesos_amb_mes_endarreriments.sort_values(by=['NumDelays'], ascending=False).head(3)
print(mesos_amb_mes_endarreriments)


# Causas d'endarreriment més freqüents 
data = df_seleccio['ArrDelay'], df_seleccio['DepDelay'], df_seleccio['CarrierDelay'], df_seleccio['WeatherDelay'], df_seleccio['NASDelay'], df_seleccio['SecurityDelay'], df_seleccio['LateAircraftDelay']
headers = ["ArrDelay", "DepDelay", "CarrierDelay", 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay']
causas = pd.concat(data, axis=1, keys=headers)
causas_mes_frequents = causas[causas > 0].count().sort_values(ascending=False).head(3)
print(causas_mes_frequents)

Columna: Month
Dades faltants: 0 

Columna: DayOfWeek
Dades faltants: 0 

Columna: ArrDelay
Dades faltants: 0 

Columna: DepDelay
Dades faltants: 0 

Columna: CarrierDelay
Dades faltants: 0 

Columna: WeatherDelay
Dades faltants: 0 

Columna: NASDelay
Dades faltants: 0 

Columna: SecurityDelay
Dades faltants: 0 

Columna: LateAircraftDelay
Dades faltants: 0 

Columna: Origin
Dades faltants: 0 

Columna: Dest
Dades faltants: 0 

Columna: Distance
Dades faltants: 0 

Columna: AirTime
Dades faltants: 0 

Columna: UniqueCarrier
Dades faltants: 0 

Columna: TotalDelay
Dades faltants: 0 

Columna: AverageSpeed
Dades faltants: 0 

Columna: Delayed
Dades faltants: 0 

count     99
unique     1
top        1
freq      99
Name: Month, dtype: object 

count     99
unique     1
top        4
freq      99
Name: DayOfWeek, dtype: object 

count     99.000000
mean      31.424242
std       47.801266
min      -26.000000
25%        5.500000
50%       19.000000
75%       45.000000
max      304.000000
Name:

/Users/claradv/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


## Exercici 3
Exporta el data set net i amb les noves columnes a Excel.

In [32]:
print(df_seleccio)

df_seleccio.to_excel('DelayedFlights_modificat.xlsx', sheet_name='Sheet1')

0  Month DayOfWeek  ArrDelay  DepDelay  CarrierDelay  WeatherDelay  NASDelay  \
1      1         4       -14         8             0             0         0   
2      1         4         2        19             0             0         0   
3      1         4        14         8             0             0         0   
4      1         4        34        34             2             0         0   
5      1         4        11        25             0             0         0   
..   ...       ...       ...       ...           ...           ...       ...   
95     1         4        32        39            32             0         0   
96     1         4        62        52            13             0        10   
97     1         4        18        23             0             0         0   
98     1         4        11        17             0             0         0   
99     1         4        34        26            18             0         8   

0   SecurityDelay  LateAircraftDelay Or